# Collision tutorial

we'll demonstrate self collision on a sawyer robot

You can see that arm of the sawyer robot collide with milk object.

## Setup
If you want to check robot's collision, install python-fcl 

And then, import CollisionManager

In [1]:
import numpy as np
import trimesh
import os

from pykin.robots.single_arm import SingleArm
from pykin.kinematics.transform import Transform
from pykin.collision.collision_manager import CollisionManager
from pykin.utils.kin_utils import apply_robot_to_scene
from pykin.utils.kin_utils import ShellColors as sc

In [2]:
file_path = 'urdf/fanuc/fanuc_r2000ic_165f.urdf'
robot = SingleArm(file_path, Transform(rot=[0.0, 0.0, 0.0], pos=[0, 0, 0]))
robot.setup_link_name("base_link", "link_6")

## Use CollisionManager

Create robot's collision manager using CollisionManager

And check self collision

In [3]:
c_manager = CollisionManager(is_robot=True)
c_manager.setup_robot_collision(robot, geom="visual")
c_manager.show_collision_info()

goal_qpos = np.array([0, 0, 0, 0, 0, 0])
robot.set_transform(goal_qpos)

******************** Robot Collision Info ********************
base_link [0. 0. 0.]
link_1 [0. 0. 0.]
link_2 [0.31200001 0.         0.67000002]
link_3 [0.31200001 0.         1.745     ]
link_4 [0.31200001 0.         1.97000003]
link_5 [1.59200001 0.         1.97000003]
link_6 [1.80700004 0.         1.97000003]
***************************************************************



In [8]:
for link, info in robot.info[c_manager.geom].items():
    if link in c_manager._objs:
        c_manager.set_transform(name=link, h_mat=info[3])

current_file_path = os.path.abspath('')
milk_path = current_file_path + "/../pykin/assets/objects/meshes/milk.stl"
test_mesh = trimesh.load_mesh(milk_path)

o_manager = CollisionManager()
o_manager.add_object("milk1", gtype="mesh", gparam=test_mesh, h_mat=Transform(pos=[1.8, 0, 2.0]).h_mat)
# o_manager.add_object("milk2", gtype="mesh", gparam=test_mesh, h_mat=Transform(pos=[0.4, 0, 0.4]).h_mat)

scene = trimesh.Scene()
scene = apply_robot_to_scene(trimesh_scene=scene, robot=robot, geom=c_manager.geom)
# scene.set_camera(np.array([np.pi/2, 0, np.pi/2]), 5, resolution=(1024, 512))

scene.add_geometry(test_mesh, node_name="milk1", transform=Transform(pos=[1.8, 0, 2.0]).h_mat)
# scene.add_geometry(test_mesh, node_name="milk2", transform=Transform(pos=[0.4, 0, 0.4]).h_mat)

'milk1'

Check collision between robot and objects

In [9]:
result, name = c_manager.in_collision_other(o_manager, return_names=True)

if result:
    print(f"{sc.FAIL}Collide!! {sc.ENDC}{list(name)[0][0]} and {list(name)[0][1]}")
scene.show(viewer='gl')

Collide!! link_6 and milk1


SceneViewer(width=1800, height=1350)